In [1]:
import tensorflow as tf

from tensorflow.keras import datasets, preprocessing, models, layers, utils, losses, optimizers

import matplotlib.pyplot as plt


In [2]:
class AlexNet:
    
    def __init__(self, input_shape, num_categories, epochs, batch_size, learning_rate, momentum):
    
        '''
        Intializes the required parameters
        '''
        self._input_shape = input_shape
        self._num_categories = num_categories
        self._epochs = epochs
        self._batch_size = batch_size
        self._learning_rate = learning_rate
        self._momentum = momentum
        
        self._model = None
        self._hist = None
        
    def create_model(self):
            
        '''
        AlexNet model defined in the paper "ImageNet Classification with Deep Convolutional Neural Networks"

        8 layers = 5 convolutional + 3 fully connected 

        Details with ImageNet data:
        224×224×3 input image 
        Conv2D(96 kernels of size 11x11x3, stride is 4) -> 55x55x96
        MaxPool(pool size of 3x3, stride is 2) -> 27x27x96
        Conv2D(256 kernels of size 5x5, pad is 2) -> 27x27x256
        MaxPool(pool size of 3x3, stride is 2) -> 13x13x256
        Conv2D(384 kernels of size 3x3, pad is 1) -> 13x13x384
        Conv2D(384 kernels of size 3x3, pad is 1) -> 13x13x384
        Conv2D(256 kernels of size 3x3, pad is 1) -> 13x13x256
        MaxPool(pool size of 3x3, stride is 2) -> 6x6x256
        FullyConnected(4096) --> 4096
        FullyConnected(4096) --> 4096
        FullyConnected(1000 softmax) --> 1000
         
        '''
        self._model = models.Sequential()
        
        
        self._model.add(layers.Conv2D(filters=96, kernel_size=(11, 11), strides=4, padding='valid', kernel_initializer='he_normal', activation='relu', input_shape=self._input_shape))
        self._model.add(layers.MaxPooling2D(pool_size=(11, 11), strides=(2, 2), padding= 'valid'))
        
        self._model.add(layers.Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding="same", kernel_initializer='he_normal', activation='relu'))
        self._model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding= 'valid'))

        self._model.add(layers.Conv2D(filters=384, kernel_size=(3, 3), padding="same", kernel_initializer= 'he_normal', activation='relu'))
        self._model.add(layers.Conv2D(filters=384, kernel_size=(3, 3), padding="same", kernel_initializer= 'he_normal', activation='relu'))
        self._model.add(layers.Conv2D(filters=256, kernel_size=(3, 3), padding="same", kernel_initializer= 'he_normal', activation='relu'))
        self._model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding= 'valid'))

        self._model.add(layers.Flatten())

        self._model.add(layers.Dense(units=4096, activation='relu'))
        self._model.add(layers.Dropout(0.5))

        self._model.add(layers.Dense(units=4096, activation='relu'))
        self._model.add(layers.Dropout(0.5))
        
        self._model.add(layers.Dense(units=1000, activation='relu'))


    def preprocess_data(self, X_train, Y_train, X_test, Y_test):
        
        X_train = X_train / 255.0 
        X_test = X_test / 255.0
        
        Y_train = utils.to_categorical(Y_train, self._num_categories)
        Y_test = utils.to_categorical(Y_test, self._num_categories)
        
        self._datagen = preprocessing.image.ImageDataGenerator(width_shift_range=0.1,
                                                               height_shift_range=0.1,
                                                               horizontal_flip=True)

        return (X_train, Y_train), (X_test, Y_test)
    
    def get_summary(self):
        return self._model.summary()
        
    def train(self, X_train, Y_train): 
        '''
        compiles and runs the model
        
        '''
        self._model.compile(optimizer=optimizers.SGD(learning_rate=self._learning_rate, momentum=self._momentum), 
                            loss=losses.CategoricalCrossentropy(), 
                            metrics=['accuracy'])
        
        self._hist = self._model.fit(self._datagen.flow(X_train, Y_train, batch_size=self._batch_size),
                                     epochs=self._epochs)
        
        return self._hist

In [3]:
alexnet = AlexNet(input_shape=(224, 224, 3), num_categories=10, epochs=20, batch_size=128, learning_rate=0.01, momentum=0.9)

In [4]:
alexnet.create_model()

In [5]:
print(alexnet.get_summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 54, 54, 96)        34944     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 256)       614656    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 384)       885120    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 384)       1327488   
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 256)       8